In [10]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

# Function to get atom positions
def get_atom_positions(molecule):
    conf = molecule.GetConformer()
    atom_positions = {}
    for atom in molecule.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        atom_positions[atom.GetIdx()] = np.array([pos.x, pos.y, pos.z])
    return atom_positions

# Function to find specific atoms by their atomic number and bonding partners
def find_atoms(molecule, atomic_num, bonded_atomic_num):
    atoms = []
    for atom in molecule.GetAtoms():
        if atom.GetAtomicNum() == atomic_num:
            for neighbor in atom.GetNeighbors():
                if neighbor.GetAtomicNum() == bonded_atomic_num:
                    atoms.append(atom)
    return atoms

# Load the molecule (for example purposes, we'll assume a simple peptide)
# Replace with actual molecule loading code
molecule = Chem.MolFromSmiles('CC(=O)NCCC(=O)NCC')
molecule = Chem.AddHs(molecule)
AllChem.EmbedMolecule(molecule)
AllChem.UFFOptimizeMolecule(molecule)

# Get all atom positions
atom_positions = get_atom_positions(molecule)

# Identify CO and NH backbone groups
co_groups = find_atoms(molecule, 8, 6)  # Oxygen bonded to Carbon
nh_groups = find_atoms(molecule, 7, 6)  # Nitrogen bonded to Carbon

# Calculate distances between all relevant pairs
def calculate_distance(atom1, atom2):
    return np.linalg.norm(atom_positions[atom1.GetIdx()] - atom_positions[atom2.GetIdx()])

# Calculate the electrostatic energy matrix
electrostatic_energy_matrix = np.zeros((len(co_groups), len(nh_groups)))

for i, co in enumerate(co_groups):
    carbon = co.GetNeighbors()[0]
    for j, nh in enumerate(nh_groups):
        nitrogen = nh.GetNeighbors()[0]

        # Distances
        r_ON = calculate_distance(co, nh)
        r_CH = min(calculate_distance(carbon, neighbor) for neighbor in nh.GetNeighbors() if neighbor.GetAtomicNum() == 1)  # Hydrogen on Carbon
        r_OH = min(calculate_distance(co, neighbor) for neighbor in nh.GetNeighbors() if neighbor.GetAtomicNum() == 1)  # Hydrogen on Oxygen
        r_CN = calculate_distance(carbon, nitrogen)

        E = 0.42 * 0.20 * 33.2 * (1/r_ON + 1/r_CH - 1/r_OH - 1/r_CN)
        electrostatic_energy_matrix[i, j] = E

/var/folders/5j/_bnmcm0j4115r6hln80zmksc0000gn/T/ipykernel_43073/2563154982.py:56: RuntimeWarning: divide by zero encountered in scalar divide
  E = 0.42 * 0.20 * 33.2 * (1/r_ON + 1/r_CH - 1/r_OH - 1/r_CN)


In [12]:
electrostatic_energy_matrix.shape

(2, 4)